In [ ]:
%matplotlib inline

# Change the seed of a DOE.

[Latin hypercube sampling](https://en.wikipedia.org/wiki/Latin_hypercube_sampling)
is an example of stochastic DOE algorithm:
given an input dimension and a number of samples,
running the algorithm twice will give two different DOEs.

For the sake of reproducibility,
GEMSEO uses a [random seed](https://en.wikipedia.org/wiki/Random_seed):
given an input dimension, a number of samples and a random seed,
running the algorithm twice will give the same DOE.

In this example,
we will see how GEMSEO uses the random seed
and how the user can change its value.


In [ ]:
from __future__ import annotations

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import create_scenario
from gemseo import execute_algo
from gemseo.algos.doe.openturns.openturns import OpenTURNS
from gemseo.algos.optimization_problem import OptimizationProblem
from gemseo.core.mdo_functions.mdo_function import MDOFunction

## At the scenario level

First,
we illustrate the use of the random seed at the [DOEScenario][gemseo.scenarios.doe_scenario.DOEScenario] level
which is the appropriate level for most users.
Then,
we will illustrate this use at the [OptimizationProblem][gemseo.algos.optimization_problem.OptimizationProblem] level
which can be useful for developers.

Let us consider an [Discipline][gemseo.core.discipline.discipline.Discipline] representing the function $y=x^2$:



In [ ]:
discipline = create_discipline("AnalyticDiscipline", expressions={"y": "x**2"})

This function is defined over the interval $[-1,1]$:



In [ ]:
design_space = create_design_space()
design_space.add_variable("x", lower_bound=-1, upper_bound=1)

We want to sample this discipline over this design space.
For that,
we express the sampling problem as a [DOEScenario][gemseo.scenarios.doe_scenario.DOEScenario]:



In [ ]:
scenario = create_scenario(
    [discipline],
    "y",
    design_space,
    scenario_type="DOE",
    formulation_name="DisciplinaryOpt",
)

and solve it:



In [ ]:
scenario.execute(algo_name="OT_OPT_LHS", n_samples=2)
scenario.formulation.optimization_problem.database.get_last_n_x_vect(2)

When using the same DOE algorithm,
solving again this problem with the same scenario leads to a new result:



In [ ]:
scenario.execute(algo_name="OT_OPT_LHS", n_samples=2)
scenario.formulation.optimization_problem.database.get_last_n_x_vect(2)

The value of the default seed was incremented
from 0 (at first execution) to 1 (at last execution),
as we can check it by setting the custom `"seed"` to 1:



In [ ]:
scenario.execute(algo_name="OT_OPT_LHS", n_samples=2, seed=1)
scenario.formulation.optimization_problem.database.get_last_n_x_vect(2)

The default seed is incremented at each execution, whatever the custom one.



## At the problem level

### Basic

Let us consider an [MDOFunction][gemseo.core.mdo_functions.mdo_function.MDOFunction] representing the function $y=x^2$:



In [ ]:
function = MDOFunction(lambda x: x**2, "f", input_names=["x"], output_names=["y"])

and defined over the unit interval $x\in[0,1]$:



In [ ]:
design_space = create_design_space()
design_space.add_variable("x", lower_bound=-1, upper_bound=1)

We want to sample this function over this design space.
For that,
we express the sampling problem as an [OptimizationProblem][gemseo.algos.optimization_problem.OptimizationProblem]:



In [ ]:
problem = OptimizationProblem(design_space)
problem.objective = function

and solve it:



In [ ]:
execute_algo(problem, algo_name="OT_OPT_LHS", algo_type="doe", n_samples=2)
problem.database.get_last_n_x_vect(2)

Note:
    We use the function [execute_algo()][gemseo.execute_algo]
    as the [OptimizationProblem][gemseo.algos.optimization_problem.OptimizationProblem] does not have an `execute()` method
    unlike the [BaseScenario][gemseo.scenarios.base_scenario.BaseScenario].

Solving again this problem with the same configuration leads to the same result:



In [ ]:
execute_algo(problem, algo_name="OT_OPT_LHS", algo_type="doe", n_samples=2)
problem.database.get_last_n_x_vect(2)

and the result is still the same if we take 1 as random seed,
as 1 is the default value of this seed:



In [ ]:
execute_algo(problem, algo_name="OT_OPT_LHS", algo_type="doe", n_samples=2, seed=1)
problem.database.get_last_n_x_vect(2)

If you want to use a different random seed,
you only have to change the value of `seed`:



In [ ]:
execute_algo(problem, algo_name="OT_OPT_LHS", algo_type="doe", n_samples=2, seed=3)
problem.database.get_last_n_x_vect(2)

### Advanced

You can also solve your problem with a lower level API
by directly instantiating the [BaseDOELIbrary][gemseo.algos.doe.base_doe_library.BaseDOELibrary] of interest.
A [BaseDOELIbrary][gemseo.algos.doe.base_doe_library.BaseDOELibrary] has a default seed generated by a [Seeder][gemseo.utils.seeder.Seeder]
that is incremented at the beginning of each execution:



In [ ]:
algo = OpenTURNS("OT_OPT_LHS")
algo.execute(problem, n_samples=2)
problem.database.get_last_n_x_vect(2)

Solving again the problem will give different samples:



In [ ]:
algo.execute(problem, n_samples=2)
problem.database.get_last_n_x_vect(2)

You can also use a specific seed instead of the default one:



In [ ]:
algo.execute(problem, n_samples=2, seed=123)
problem.database.get_last_n_x_vect(2)